#**Name: Abdelrahman Sayed Abdallah**
#**ID: 20180152**
# **GROUP: S6**

In [64]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
# Collection of documents (corpus)

review_0 = "The Glider. II is the best soccer ball"
review_1 = "What a bad soccer ball"
review_2 = "I am happy with The glider"
review_3 = "messi is the best soccer ball player in the world"
review_4 = "economics is a needed science for all world countries to improve their states"
review_5 = "car racing returns improve the national income"

In [66]:
def clean_description(doc):
    doc=doc.translate(str.maketrans('','',string.punctuation))
    doc=doc.lower()
    doc=doc.split()
    doc=[word for word in doc if not word in stopwords.words()]
    doc=' '.join(doc)
    return doc
    

  
    
    
  

In [67]:
doc0=clean_description(review_0)
doc1=clean_description(review_1)
doc2=clean_description(review_2)
doc3=clean_description(review_3)
doc4=clean_description(review_4)
doc5=clean_description(review_5)
docs= [doc0,doc1, doc2, doc3,doc4,doc5]

In [68]:
docs

['glider best soccer ball',
 'bad soccer ball',
 'happy glider',
 'messi best soccer ball player world',
 'economics needed science world countries improve states',
 'car racing returns improve national income']

In [69]:
# Construct an inverted index
# here as a Python dictionary for ease of interpretability

inverted_index = {}

for i, doc in enumerate(docs):
    for term in doc.split():
        if term in inverted_index:
            inverted_index[term].add(i)
        else: inverted_index[term] = {i}

inverted_index

{'bad': {1},
 'ball': {0, 1, 3},
 'best': {0, 3},
 'car': {5},
 'countries': {4},
 'economics': {4},
 'glider': {0, 2},
 'happy': {2},
 'improve': {4, 5},
 'income': {5},
 'messi': {3},
 'national': {5},
 'needed': {4},
 'player': {3},
 'racing': {5},
 'returns': {5},
 'science': {4},
 'soccer': {0, 1, 3},
 'states': {4},
 'world': {3, 4}}

In [70]:
posting_list = inverted_index['soccer']
posting_list

{0, 1, 3}

In [71]:
def or_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            result.append(posting2[p2])
            p2 += 1
        else:
            result.append(posting1[p1])
            p1 += 1
    while p1 < len(posting1):
        result.append(posting1[p1])
        p1 += 1
    while p2 < len(posting2):
        result.append(posting2[p2])
        p2 += 1
    return result

In [72]:
pl_1 = list(inverted_index['soccer'])
pl_2 = list(inverted_index['glider'])
or_postings(pl_1, pl_2) 

[0, 1, 2, 3]

In [73]:
def and_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            p2 += 1
        else:
            p1 += 1
    return result

In [74]:
def Not_operation(posting1, posting2):
    result = list()
    for i in posting1:
      if i not in posting2:
          result.append(i)
    return result

In [75]:
pl_3 = list(inverted_index['ball'])
pl_4= list(inverted_index['bad'])

and_postings(pl_1, pl_2)

[0]

In [76]:
#query_with_one_boolean_operator
first='soccer or ball'
second='soccer not glider'
third='ball and glider'
fourth='soccer and ball'

In [77]:
def return_result(query_with_one_boolean_operator):
  query_with_one_boolean_operator=query_with_one_boolean_operator.split()
  for i in range(len(query_with_one_boolean_operator)):
    if query_with_one_boolean_operator[i]=='and' or query_with_one_boolean_operator[i]=='or' or query_with_one_boolean_operator[i]=='not':
      pl_1=list(inverted_index[query_with_one_boolean_operator[i-1]])
      pl_2=list(inverted_index[query_with_one_boolean_operator[i+1]])
      if query_with_one_boolean_operator[i]=='or':
        result=or_postings(pl_1, pl_2)
        return result
      if query_with_one_boolean_operator[i]=='and':
        result=and_postings(pl_1, pl_2)
        return result
      if query_with_one_boolean_operator[i]=='not':
        result=Not_operation(pl_1, pl_2)
        return result


In [78]:
soccer_or_ball=return_result(first)#should return 0,1,3
soccer_or_ball

[0, 1, 3]

In [79]:
soccer_and_not_glider=return_result(second)#should return 1,3
soccer_and_not_glider

[1, 3]

In [80]:
ball_and_glider=return_result(third)#should return 0
ball_and_glider

[0]

In [81]:
soccer_and_ball=return_result(fourth)#should return 0,1,3
soccer_and_ball

[0, 1, 3]

In [82]:
#query_with_more_than_one_boolean_operator
fifth='(soccer and bad) or messi'
sixth='player and world and not economics'
seventh='car or glider or science'
eight='best and soccer and ball'

In [83]:
fifth=return_result('soccer and ball')
fifth=or_postings(fifth,list(inverted_index['messi']))#should return 0,1,3

In [84]:
fifth

[0, 1, 3]

In [85]:
sixth=return_result('player and world')
sixth=Not_operation(sixth,list(inverted_index['economics']))#should return 3

In [86]:
sixth

[3]

In [87]:
seventh=return_result('car or glider')
seventh=or_postings(seventh,list(inverted_index['science']))#should return 0,2,4,5

In [88]:
seventh

[0, 2, 4, 5]

In [89]:
eight=return_result('best and soccer')
eight=and_postings(eight,list(inverted_index['ball']))#should return 0,3

In [90]:
eight

[0, 3]

# ***THANK YOU***